In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from skimage.filters import threshold_otsu
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
import canny_edge_detector as canny
from sobel import show_images
import matplotlib.pyplot as plt 
import skimage.io as io
import os

import numpy as np
IMG_PATH = 'images/kelvinl1.bmp'
o_circles,inner_circles = canny.process_for_daugman(canny,IMG_PATH)
print(o_circles,inner_circles)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
7 240 320 54
dsf  4
63 8 8 8
dsf  4
42 8 8 55
dsf  4
[(152.0, 122.0, 61)] [(151.0, 121.0, 26)]
